In [1]:
# %matplotlib widget

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table

from lvmdrp import path, __version__ as drpver
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp

from lvmdrp.core.tracemask import TraceMask
from lvmdrp.functions import imageMethod

ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")
MASTER_CON_LAMPS = {"b": "ldls", "r": "ldls", "z": "quartz"}
MASTER_ARC_LAMPS = {"b": "hgne", "r": "neon", "z": "neon"}


f = np.sqrt(2 * np.pi)
def gaussians(pars, x):
    y = pars[0][:, None] * np.exp(-0.5 * ((x[None, :] - pars[1][:, None]) / pars[2][:, None]) ** 2) / (pars[2][:, None] * f)
    return np.sum(y, axis=0)

In [2]:
frames_table = md.get_metadata(tileid="*", mjd=60177)
frames_table.query("imagetyp == 'bias' | imagetyp == 'dark' | imagetyp == 'flat' | imagetyp == 'arc'", inplace=True)

[INFO]: loading metadata store at /home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/raw_metadata.hdf5
[INFO]: found 540 frames in store '/home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/raw_metadata.hdf5'
[INFO]: number of frames after filtering 540
[INFO]: total number of frames found 540


In [3]:
masters_mjd = frames_table.mjd.min()
masters_path = os.path.join(ORIG_MASTER_DIR, f"{masters_mjd}")

In [4]:
masters_flat = md.get_metadata(kind="master", imagetyp="flat").query("mjd == @masters_mjd").sort_values("camera")
masters_flat

[INFO]: loading metadata store at /home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/master_metadata.hdf5
[INFO]: found 120 frames in store '/home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/master_metadata.hdf5'
[INFO]: filtering by imagetyp = 'flat'
[INFO]: number of frames after filtering 18
[INFO]: total number of frames found 18


,tileid,mjd,rmjd,imagetyp,spec,camera,exptime,neon,hgne,krypton,...,argon,ldls,quartz,quality,qual,stage,status,drpqual,nframes,name
12,1111,60177,60177,flat,sp1,b1,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,2,lvm-mflat_ldls-b1
14,1111,60177,60177,flat,sp2,b2,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,2,lvm-mflat_ldls-b2
10,1111,60177,60177,flat,sp3,b3,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,2,lvm-mflat_ldls-b3
16,1111,60177,60177,flat,sp1,r1,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,2,lvm-mflat_ldls-r1
11,1111,60177,60177,flat,sp2,r2,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,2,lvm-mflat_ldls-r2
15,1111,60177,60177,flat,sp3,r3,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,2,lvm-mflat_ldls-r3
13,1111,60177,60177,flat,sp1,z1,10.0,False,False,False,...,False,False,True,excellent,0,1,0,0,2,lvm-mflat_quartz-z1
9,1111,60177,60177,flat,sp2,z2,10.0,False,False,False,...,False,False,True,excellent,0,1,0,0,2,lvm-mflat_quartz-z2
17,1111,60177,60177,flat,sp3,z3,10.0,False,False,False,...,False,False,True,excellent,0,1,0,0,2,lvm-mflat_quartz-z3


In [5]:
for flat in masters_flat.to_dict("records"):
    camera = flat["camera"]
    
    lamp = MASTER_CON_LAMPS[camera[0]]
    if lamp == "ldls":
        counts_threshold = 20
    elif lamp == "quartz":
        counts_threshold = 1000
    
    # define master continuum exposure path
    mmod_path = os.path.join(masters_path, f"lvm-mmod-{camera}.fits")
    mtrace_path = os.path.join(masters_path, f"lvm-mtrace-{camera}.fits")
    mwidth_path = os.path.join(masters_path, f"lvm-mwidth-{camera}.fits")
    maper_path = os.path.join(masters_path, f"lvm-apercorr-{camera}.fits")

    if os.path.isfile(maper_path):
        print(f"skipping {maper_path}, file already exist")
        continue

    # load frame and traces
    img = imageMethod.loadImage(mmod_path)
    img._error = None
    img._mask[...] = False
    # img.replaceMaskMedian(5,5)
    trace_cent = TraceMask()
    trace_cent.loadFitsData(mtrace_path)
    trace_cent._mask[...] = False
    trace_fwhm = TraceMask()
    trace_fwhm.loadFitsData(mwidth_path)
    trace_fwhm._mask[...] = False

    # extract aperture and optimal spectra
    data_aper, error_aper, mask_aper = img.extractSpecAperture(trace_cent, aperture=3)
    data_fit, error_fit, mask_fit = img.extractSpecOptimal(trace_cent, trace_fwhm, plot_fig=False)

    # save aperture correction
    new_mask = np.zeros_like(data_fit)
    aper_corr = imageMethod.Image(data=data_fit/data_aper, header=img._header, mask=new_mask)
    aper_corr.writeFitsData(maper_path)